In [1]:
import json
import re
from tqdm import tqdm
import random

## load raw phonedb dataset scraped in phonedb_scraper.ipynb

In [2]:


with open('phonedb_dataset.json', 'r') as f:
    phonedb_data = json.load(f)

model_long_names = list(phonedb_data.keys())
#print(phonedb_data[model_long_names[0]][0].replace("\\n", "\n"))

brand_name_indexer = {}

for lname in model_long_names:
    brand_name = lname.split()[0]
    #print(brand_name)
    if brand_name not in brand_name_indexer.keys():
        brand_name_indexer[brand_name] = []
    brand_name_indexer[brand_name].append(lname)
print(brand_name_indexer.keys())
#print(brand_name_indexer[bn])

dict_keys(['Samsung', 'Xiaomi', 'Huawei', 'Apple'])


## Create phone database 

The database is simply a python dictionary, which uses short abbreviated phone model name as keys, and a value in the database, indexed by the keys, is a python list containing the html text data for all long model names associated with the short name used as the key.

The short abbr. names are gotten from regex filters using human prior knowledge rules on the naming convention for each phone brand.

Example short name: iPhone 13

Example long names associated with the above short name: 
* Apple iPhone 13 mini 5G A2629 Dual SIM TD-LTE CN 512GB 
* Apple iPhone 13 mini 5G A2629 Dual SIM TD-LTE CN 256GB
* ......
* Apple iPhone 13 5G A2634 Dual SIM TD-LTE CN 128GB 
* ......
* Apple iPhone 13 Pro 5G A2639 Dual SIM TD-LTE CN 1TB
* ......

In [3]:
phone_name_abbr = {}

In [4]:
bn = 'Apple'
phone_name_abbr[bn] = set()
for lname in brand_name_indexer[bn]:
    result = re.search(r'^(.*?)\bA\d{4}\b', lname)
    if result != None:
        sname = result.group(1).replace(bn, "").replace("5G", "").strip()
    phone_name_abbr[bn].add(sname)
    
# nls = list(phone_name_abbr[bn])
# nls.sort()
# for n in nls:
#     print(n)

In [5]:
bn = 'Huawei'
phone_name_abbr[bn] = set()
remainder_set = set()
sname = None
for lname in brand_name_indexer[bn]:
    result = re.search(r'(.*?)(?i)edition', lname)
    if result != None:
        #sname = result.group(0)
        sname_split = result.group(0).split()
        sname = ' '.join(sname_split[:-2])
        result = re.search(r'(.*?)(?i)Lite', sname)
        if result != None:
            sname = result.group(0)
        #print(lname, ": ", sname)
    else:
        result = re.search(r'(.*?)(?i)(\s4G|\s5G)', lname)
        if result != None:
            sname = result.group(0)
        else:
            remainder_set.add(lname)
    if sname != None and 'Tablet' not in sname:
        if 'Premium' not in sname and 'Dual' not in sname:
            sname = sname.replace("5G", "").replace("4G", "")
            phone_name_abbr[bn].add(sname)

remainder_set2 = set() 
for lname in list(remainder_set):
    result = re.search(r'(.*?)(?i)Dual', lname)
    if result != None:
        sname = result.group(1)
        phone_name_abbr[bn].add(sname)
    else:
        remainder_set2.add(lname)
    
# nls = list(remainder_set)
# nls.sort()
# for n in nls:
#     print(n)
    
    
# nls = list(phone_name_abbr[bn])
# nls.sort()
# for n in nls:
#     print(n)

/tmp/ipykernel_764/4202839765.py:6: DeprecationWarning: Flags not at the start of the expression '(.*?)(?i)edition' but at position 5
  result = re.search(r'(.*?)(?i)edition', lname)
/tmp/ipykernel_764/4202839765.py:16: DeprecationWarning: Flags not at the start of the expression '(.*?)(?i)(\\s4G|\\s5G)' but at position 5
  result = re.search(r'(.*?)(?i)(\s4G|\s5G)', lname)
/tmp/ipykernel_764/4202839765.py:11: DeprecationWarning: Flags not at the start of the expression '(.*?)(?i)Lite' but at position 5
  result = re.search(r'(.*?)(?i)Lite', sname)
/tmp/ipykernel_764/4202839765.py:28: DeprecationWarning: Flags not at the start of the expression '(.*?)(?i)Dual' but at position 5
  result = re.search(r'(.*?)(?i)Dual', lname)


In [6]:
bn = 'Xiaomi'
phone_name_abbr[bn] = set()
remainder_set = set()
sname = None
for lname in brand_name_indexer[bn]:
    result = re.search(r'(.*?)(?i)edition', lname)
    if result != None:
        #sname = result.group(0)
        sname_split = result.group(0).split()
        sname = ' '.join(sname_split[:-2])
        result = re.search(r'(.*?)(?i)Lite', sname)
        if result != None:
            sname = result.group(0)
        #print(lname, ": ", sname)
    else:
        result = re.search(r'(.*?)(?i)(\s4G|\s5G)', lname)
        if result != None:
            sname = result.group(0)
        else:
            remainder_set.add(lname)
    if sname != None and 'Tablet' not in sname:
        if 'Premium' not in sname and 'Dual' not in sname:
            sname = sname.replace("5G", "").replace("4G", "")
            phone_name_abbr[bn].add(sname)

remainder_set2 = set() 
for lname in list(remainder_set):
    result = re.search(r'(.*?)(?i)Dual', lname)
    if result != None:
        sname = result.group(1)
        phone_name_abbr[bn].add(sname)
    else:
        remainder_set2.add(lname)
    
# nls = list(remainder_set2)
# nls.sort()
# for n in nls:
#     print(n)
    
    
# nls = list(phone_name_abbr[bn])
# nls.sort()
# for n in nls:
#     print(n)

In [7]:
bn = 'Samsung'
phone_name_abbr[bn] = set()
remainder_set = set()
sname = None
for lname in brand_name_indexer[bn]:
    result = re.search(r'(.*?)(?i)edition', lname)
    if result != None:
        #sname = result.group(0)
        sname_split = result.group(0).split()
        sname = ' '.join(sname_split[:-2])
        result = re.search(r'(.*?)(?i)Lite', sname)
        if result != None:
            sname = result.group(0)
        #print(lname, ": ", sname)
    else:
        result = re.search(r'(.*?)(?i)(\s4G|\s5G)', lname)
        if result != None:
            sname = result.group(0)
        else:
            remainder_set.add(lname)
    if sname != None and 'Tablet' not in sname:
        if 'Premium' not in sname and 'Dual' not in sname:
            sname = sname.replace("5G", "").replace("4G", "")
            sname = re.sub(r'(\bSM-\w+)|(\b/DS\b)|(\b\d{4})', '', sname)
            sname = sname.replace("  ", " ")
            phone_name_abbr[bn].add(sname)

remainder_set2 = set() 
for lname in list(remainder_set):
    result = re.search(r'(.*?)(?i)Dual', lname)
    if result != None:
        sname = result.group(1)
        sname = re.sub(r'(\bSM-\w+)|(\b/DS\b)|(\b\d{4})', '', sname)
        sname = sname.replace("  ", " ")
        phone_name_abbr[bn].add(sname)
    else:
        remainder_set2.add(lname)

    
# nls = list(remainder_set)
# nls.sort()
# for n in nls:
#     print(n)
    
    
# nls = list(phone_name_abbr[bn])
# nls.sort()
# for n in nls:
#     print(n)

In [8]:
name_list = []
for bn in phone_name_abbr.keys():
    for n in phone_name_abbr[bn]:
        name_list.append(n.strip().replace("  ", " "))

In [9]:
long_name_list = model_long_names
print(len(name_list))
print(len(long_name_list))

394
1064


In [10]:
name_map = {}

for bn in brand_name_indexer.keys():
    #print(bn)
    short_name_dict = {}
    
    bn2 = bn
    if bn == 'Apple':
        bn2 = 'iPhone'
    for sn in ([s for s in name_list if bn2 in s]):
        
        sn2 = sn
        if 'Samsung' in sn2:
            sn2 = sn.replace('Samsung', '').strip()
        lns = []
        
        for ln in ([l for l in long_name_list if sn2 in l]):
             lns.append(ln)
                
        short_name_dict[sn] = lns
        
    name_map.update(short_name_dict)

#list(name_map.keys())
#name_map

In [11]:
# Helper function for easier usage
def query_specs_list(short_name, debug=False, replace_new_line = True):
    spec_list = []
    for ln in name_map[short_name]:
        if debug:
            print(ln)
        if replace_new_line:
            spec = phonedb_data[ln][0].replace("\\n", "\n")
        else:
            spec = phonedb_data[ln][0]
        spec_list.append(spec)
    return spec_list

In [12]:
exception_list = []
for i in range(0, len(name_list) - 1):
    query_name = name_list[i]
    spec_list = query_specs_list(query_name, debug=False)
    if len(spec_list) == 0: 
        exception_list.append(query_name)
print(exception_list)        
print(len(exception_list))




['iPhone SE 2022 3rd gen', 'Huawei Honor 10 +', 'Huawei Honor 8X Max +', 'Xiaomi Black Shark 4S Gundam', 'Xiaomi Mi Mix Fold 2021 Ceramic', 'Xiaomi Redmi K30 Extreme', 'Xiaomi Civi 2 Hello Kitty', 'Samsung Galaxy Tab S8 Ultra 14.6', 'Samsung Galaxy Tab S8+ 12.4', 'Samsung Galaxy Z Fold2 Thom', 'Samsung Galaxy Z Fold3 Thom', 'Samsung Galaxy Tab S8 11', 'Samsung Galaxy Z Flip 4 Maison', 'Samsung Galaxy Z Flip 3 Thom Browne']
14


In [13]:
name_list = [item for item in name_list if item not in exception_list]

# print(len(name_map.keys()))
# print(len(name_list))
keys = list(name_map.keys())
for key in keys:
    if key not in name_list:
        del name_map[key]
# print(len(name_map.keys()))
# print(len(name_list))

## Example Usage

In [14]:
#Example Usage

### name_list stores all short names
### name_list is equivalent to the keys of name_map (called within the query_specs_list func)

random_idx = random.randint(0, len(name_list) - 1)
query_name = name_list[random_idx]
spec_list = query_specs_list(query_name)

print(spec_list[0][0:500]) #only print the first text in the list in its first 500 letters

Brand: Samsung 
 Model: SM-T735C Galaxy Tab S7 FE 4G 12.4 2021 TD-LTE CN 64GB 
 Brief Info: With low latency and a soft tip, the included S Pen offers a natural writing experience. Chinese LTE variant of Tab S7 FE with 4 GiB RAM, 64 GB ROM 
 Released: 2021 Jun 14 
 Announced: 2021 May 26 
 Hardware Designer: Samsung Electronics 
 Manufacturer: Samsung Electronics 
 Codename: Samsung T730 
 General Extras: Active stylus 
 Device Category: Tablet 
 Width: 284.8 mm, 11.21 inch 
 Height: 185 mm, 7.2


## Store results in a pickle file

The results are stored in a file named "phone_dataset.pkl" (Although it's better to rename it as phone_database.pkl, reruning other long-running programs which use this file are very time-consuming, so just keep it as it is)

To use this database, another python file only need phonedb_data, name_map, and copy paste the "query_specs_list" helper function in an above cell.


In [15]:
# Store stuff
import pickle
with open("phone_dataset.pkl", "wb") as f:
    pickle.dump([phonedb_data, name_map], f)

In [17]:
# Load stuff
with open("phone_dataset.pkl", "rb") as f:
    pdb = pickle.load(f)
    
pd, name_map = pdb